# A multi-compartmental model of a pyramidal neuron

 

## Step 1: Setup

In [ ]:
# Setup inline plotting
#%matplotlib widget
%matplotlib inline
import matplotlib.pyplot as plt
#import matplotlib
#matplotlib.rcParams['path.simplify'] = False

In [ ]:
# For Google Colab, this line installs NEURON
!pip install neuron quantities neurom 

In [ ]:
# We will let this library handle unit conversion for us
import quantities as pq
from quantities import um, nS, mV, cm, ms, nA, S, uF, Hz, degrees, s, MOhm, mS, mm, Ohm

In [ ]:
# Import and initialize NEURON
import neuron
from neuron import h
h.load_file("stdrun.hoc")
h.load_file("import3d.hoc")
#h.load_file("stdlib.hoc") # need geom_nseg

In [ ]:
# Import other modules we need
import numpy as np
from tqdm import tqdm
import time

In [ ]:
import neurom as nm

In [ ]:
from neurom.view import matplotlib_impl, matplotlib_utils

In [ ]:
morph = nm.load_morphology('data/morphology.asc')

In [ ]:
# instead of: viewer.draw(m)
matplotlib_impl.plot_morph(morph)
plt.xlabel(r"x ($\rm{\mu m}$)") 
plt.ylabel(r"y ($\rm{\mu m}$)") 

In [ ]:
# instead of: viewer.draw(m, mode='3d')
#matplotlib_impl.plot_morph3d(morph)

In [ ]:
# instead of: viewer.draw(m, mode='dendrogram')
matplotlib_impl.plot_dendrogram(morph)

## Step 2: Define the circuit


In [ ]:
# constants
h.dt = 0.025
h.celsius = 34

In [ ]:
def rerun_sim(tstop, tskip=0, v_init=-84.0, viz=False, steps=50, framerate=None):
    if framerate:
        dt = 1.0/framerate
    else:
        dt = None
    h.finitialize( float(v_init) )
    h.continuerun( float(tskip * ms) )
    for t in tqdm(np.linspace(tskip,tstop,steps)):
        t1 = time.time()
        h.continuerun( float(t * ms) )
        if viz==True:
            sp.flush()
        t2 = time.time()
        if dt:
            wait = dt-(t2-t1)
            if wait > 0:
                time.sleep(wait)

In [ ]:
class MyCell:
    def __init__(self):
        """ Create the neuron, load the morphology, replace the axon with a stub, 
        configure nseg for each compartment
        """
        # This idiom below imports the morphology file and instaniates it as h.Sections
        morph_reader = h.Import3d_Neurolucida3()
        morph_reader.input('data/morphology.asc')
        i3d = h.Import3d_GUI(morph_reader, 0)
        i3d.instantiate(self)
        
        self.replace_axon()
        self.geom_nseg()
        self.check_diameters()
        
        # Alias for readability 
        self.basal = self.dend
        self.apical = self.apic
        self.axonal = self.axon
        
    def geom_nseg(self):
        """ Setup the nseg parameter for all sections
        Results in about 2 compartments per 40um
        """
        for sec in self.soma[0].wholetree():
            sec.nseg = int((sec.L/(0.1*h.lambda_f(2500))+.9)/2)*2 + 1
            #sec.nseg = int(sec.L/40)*2 + 1
    
    def check_diameters(self):
        """ Check there are no small diameter sections"""
        for sec in self.dend:
            if sec.diam<0.001:
                raise ValueError("diam=%f at section '%s'" % (sec.diam, sec.name()))
        
        for sec in self.apic:
            if sec.diam<0.001:
                raise ValueError("diam=%f at section '%s'" % (sec.diam, sec.name()))                    
    
    def replace_axon(self):
        """ We will not simulate the axon, 
        so this function deletes it and replaces it with a stub with same diameter """
        # Find the axonal section that connects to the soma
        axon_root = [x for x in self.soma[0].children() if x in self.axon]
        # There should be only one for this morphology
        assert len(axon_root)==1
        axon_root = axon_root[0]
        
        # Get diameter
        d = axon_root(0.0).diam
        
        # Delete all axon sections
        for sec in self.axon:
            h.delete_section(sec=sec)
        
        # Create new axon
        axon = h.Section()
        self.axon = [axon]
        axon.L = 60                                                              
        axon.diam = d   
        
        # Connect it to the some
        axon.connect(self.soma[0](1.0),0.0)
        

In [ ]:
m = MyCell()

In [ ]:
print('%d apic sections' % len(m.apic))
print('%d basal sections' % len(m.dend))
print('%d somatic sections' % len(m.soma))
print('%d axonal sections' % len(m.axon))

In [ ]:
# An alias for the soma
soma = m.soma[0]

#### Count the number of sections and segments

In [ ]:
num_segments = sum([d.nseg for d in soma.wholetree()])

In [ ]:
num_segments

In [ ]:
num_sections =0
for sec in soma.wholetree():
    num_sections+=1

In [ ]:
num_sections  

#### Visualize the neuron within NEURON!

In [ ]:
# Only works if Jupyter & NEURON are running locally, I imagine.
#from neuron import gui

sp = h.PlotShape()


In [ ]:
sp.show(1)
sp.show(0)  # show diameters

### Query NEURON for soma.L, soma.diam, area and volume

In [ ]:
soma.L, soma.diam

In [ ]:
volume = soma(0.5).volume() * um**3

In [ ]:
area = soma(0.5).area() * um**2

In [ ]:
area

In [ ]:
volume

### Assign the membrane capacitance, resistivity and passive properties "everywhere" 

In [ ]:
h.units("cm")  # Query the expected units

In [ ]:
h.units("Ra")

In [ ]:
specific_membrane_capacitance = 1 * uF/cm**2

In [ ]:
# This will look over sections in the whole tree (axons, dendrites, apical dendrites) connected/attached to the soma 
for sec in soma.wholetree():
    sec.insert("pas")
    sec.e_pas = -75.0
    sec.g_pas = 3e-5
    sec.cm = specific_membrane_capacitance #  specific membrane capacitance (micro Farads / cm^2)
    sec.Ra = 100 * Ohm*cm # resistivity 

In [ ]:
# This is to compendate for spines (which we don't model explicitly, but effect capacitance considerably)
for sec in m.basal+m.apical:
    sec.cm = 2

### Assign the channels in the various regions of the neuron

In [ ]:
for sec in m.basal+m.apical+m.soma:
    sec.insert("Ih")

In [ ]:
# Na and K channels
for sec in m.apical:
    sec.insert("Im")

for sec in m.soma+m.apical:
    sec.insert("NaTs2_t")
    
for sec in m.axonal:
    sec.insert("Nap_Et2")
    sec.insert("K_Pst")
    sec.insert("K_Tst")
    sec.insert("NaTa_t")
    
for sec in m.axonal+m.soma+m.apical:
    sec.insert("SKv3_1")       

In [ ]:
# Calcium and SK channels
for sec in m.soma+m.axonal:
    sec.insert("Ca_HVA")
    sec.insert("SK_E2")
    sec.insert("Ca_LVAst")
    sec.insert("CaDynamics_E2")   

### Parameterize the channel conductances
Note, these values come from an external data-driven optimization process to fit neuronal firing features to experimental data <br>
See: https://www.frontiersin.org/articles/10.3389/fninf.2016.00017/full

#### Define and apply an exponential distribution of Ih channel conductances in the apical dendrite

In [ ]:
def distribute_Ih(x):
    """Spatial specific conductance distribution of Ih channels
    x (in um) is the path distance distance to the soma
    """
    from math import exp
    return 6.9568e-5*(2.4*exp(x*0.003100) - 1)

In [ ]:
for sec in m.apical:
    for seg in sec:
        x = h.distance(soma(0.0),seg)
        seg.gIhbar_Ih = distribute_Ih(x)

#### All other channels have constant conductances

In [ ]:
for sec in m.basal+m.soma:
    sec.gIhbar_Ih = 8e-5

In [ ]:
for sec in m.apical:
    sec.gNaTs2_tbar_NaTs2_t = 0.026145
    sec.gSKv3_1bar_SKv3_1 = 0.004226
    sec.gImbar_Im = 0.000143

In [ ]:
for sec in m.axonal:
    sec.gNaTa_tbar_NaTa_t = 3.137968
    sec.gK_Tstbar_K_Tst = 0.089259
    sec.gamma_CaDynamics_E2 = 0.002910
    sec.gNap_Et2bar_Nap_Et2 = 0.006827
    sec.gSK_E2bar_SK_E2 = 0.007104
    sec.gCa_HVAbar_Ca_HVA = 0.000990
    sec.gK_Pstbar_K_Pst = 0.973538
    sec.gSKv3_1bar_SKv3_1 = 1.021945
    sec.decay_CaDynamics_E2 = 287.198731
    sec.gCa_LVAstbar_Ca_LVAst = 0.008752

In [ ]:
for sec in m.soma:
    sec.gamma_CaDynamics_E2 = 0.000609
    sec.gSKv3_1bar_SKv3_1 = 0.303472
    sec.gSK_E2bar_SK_E2 = 0.008407
    sec.gCa_HVAbar_Ca_HVA = 0.000994
    sec.gNaTs2_tbar_NaTs2_t = 0.983955
    sec.decay_CaDynamics_E2 = 210.485284
    sec.gCa_LVAstbar_Ca_LVAst = 0.000333

In [ ]:
for sec in m.soma+m.apical+m.axon:
    sec.ena = 50
    sec.ek = -85

### Add soma current pulse

In [ ]:
step_amp = [0.4299372, 0.4657653, 0.5015934]
hyp_amp = -0.193481 

In [ ]:
stim = h.IClamp(soma(0.5))

In [ ]:
stim.delay = 500 * ms
stim.dur = 1000 * ms
stim.amp = step_amp[0]

In [ ]:
hyp_iclamp = h.IClamp(soma(0.5))

In [ ]:
hyp_iclamp.delay = 0 * ms
hyp_iclamp.dur = 5000 * ms  
hyp_iclamp.amp = hyp_amp * nA

## Step 3: Run the simulation

### Define recordings of simulation variables

In [ ]:
rec_v = h.Vector().record(soma(0.5)._ref_v)
rec_t = h.Vector().record(h._ref_t)

In [ ]:
rerun_sim(tstop=1500, tskip=500, viz=True, steps=1000, framerate=20)
# For Shape plot viz

In [ ]:
sp.color?

In [ ]:
#rerun_sim(tstop=2000, steps=1000)

In [ ]:
vs = []
for amp in step_amp:
    stim.amp = amp
    rerun_sim(tstop=2000, steps=1000)
    vs.append(np.array(rec_v))

## Step 4: Plot the results

In [ ]:
plt.figure()
plt.plot(rec_t, vs[0], label="py version")

In [ ]:
plt.figure()
plt.plot(rec_t, vs[1], label="py version")

In [ ]:
plt.figure()
plt.plot(rec_t, vs[2], label="py version")